In [2]:
import torch


def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_device_name():
    return torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu"

def get_device_count():
    return torch.cuda.device_count() if torch.cuda.is_available() else 0

def get_device_memory():
    return torch.cuda.get_device_properties(0).total_memory if torch.cuda.is_available() else 0

def get_device_memory_allocated():
    return torch.cuda.memory_allocated() if torch.cuda.is_available() else 0

print( torch.__version__ )
print( get_device() )
print( get_device_name() )
print( get_device_count() )
print( get_device_memory() )
print( get_device_memory_allocated() )


2.2.2+cpu
cpu
cpu
0
0
0


In [4]:
import abc

class AbstractLayer(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def output(self, input):
        """Compute the activation values of the neurons in this Layer."""
        raise NotImplementedError("Must override output method")



    @abc.abstractmethod
    def reset(self):
        """Reset the layer."""
        raise NotImplementedError("Must override reset method")

In [ ]:
#create a class that inherits from the abstract class and the torch.nn.module
class BaseLayer(AbstractLayer, torch.nn.Module):
    def __init__(self, params: Dict, name: str, node_no: int, transfer: Callable):
        super(BaseLayer, self).__init__()
        self.params = params
        self.name = name
        self.node_no = node_no
        self.transfer = transfer
        self.inputs = None
        self.outputs = None
        self.biases = torch.nn.Parameter(self.params['BIAS_INIT'](self.node_no), name='B_%s' %(self.name,), trainable=True)
    
    def __rshift__(self, other ):
        print( other )
    
    def output( self, input ):
        print( self.name, self.inputs.output(input).shape, self.biases.shape )
        return self.transfer( torch.matmul( self.inputs.output(input), self.biases ) )
    
    def update( self, optimizer ):
        # update the biases
        gradient = self.get_gradient()
        optimizer.step(gradient)

    def get_gradient(self):
        return torch.autograd.grad(self.outputs, self.biases, retain_graph=True)[0]
    
    def reset(self):
        self.biases = torch.nn.Parameter(self.params['BIAS_INIT'](self.node_no), name='B_%s' %(self.name,), trainable=True)

In [ ]:
class Layer(BaseLayer):
    def __init__(self, params: Dict, name: str, node_no: int, transfer: Callable):
        super(Layer, self).__init__(params, name, node_no, transfer)
    
    def __rshift__( self, other ):
        return NN( self, other )